In [1]:
# True: Google Colab Notebook
# False: My local PC
colab = False
if colab: 
    from google.colab import drive
    drive.mount('/content/drive')
    !ls /content/drive/MyDrive/output/otto/
    base_path = '/content/drive/MyDrive'
    notebook_path = base_path + '/otto/notebook'
    !pip3 install optuna
else:
    base_path = '../data'
    notebook_path = '.'

# Preprocessing

In [2]:
# ====================================================
# Library
# ====================================================
import gc
import warnings
warnings.filterwarnings('ignore')
import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
import itertools
from imblearn.under_sampling import RandomUnderSampler
import sys
sys.path.append(f"{notebook_path}/../src/")
import feature_engineering as fe

In [3]:
#train = pd.read_parquet('/content/drive/MyDrive/output/otto/train_50.parquet')
#train = pd.read_parquet(f'{base_path}/output/otto/train_50_tmp.parquet')
#train = pd.read_parquet(f'{base_path}/output/otto/train_50_0.parquet') # 0.592
train = pd.read_parquet(f'{base_path}/output/otto/train_50_0_ver2.parquet')

In [4]:
DEBUG_MODE = False
OPTUNA_FLAG = False
CROSS_TARGET_STACKING = True

if DEBUG_MODE:
    train = train.head(100000)
IGNORE_COL_ID = ['session','aid']

#TYPE_MODE = 'clicks'
#TYPE_MODE = 'carts'
TYPE_MODE = 'orders'
IGNORE_COL_TARGET = ['y_clicks', 'y_carts', 'y_orders']


if TYPE_MODE == 'clicks':
    target = 'y_clicks'
    # under sampling 1.3 -> 2.5%
    #pos_neg_ratio = 1/39
    pos_neg_ratio = 1/29 # 3.3%
    # used for cross target stacking
    cross_target_list = ['carts', 'clicks']
elif TYPE_MODE == 'carts':
    target = 'y_carts'
    # under sampling 1.6 -> 2.5%
    pos_neg_ratio = 1/39
    # used for cross target stacking
    cross_target_list = ['orders', 'clicks']
elif TYPE_MODE == 'orders':
    target = 'y_orders'
    # under sampling 2.1 -> 2.5%
    pos_neg_ratio = 1/39
    # used for cross target stacking
    cross_target_list = ['carts', 'clicks']

session_path = f'{base_path}/output/otto/valid_session_features.parquet'
aid_path = f'{base_path}/output/otto/valid_aid_features.parquet'

In [5]:
# 負例しかないものは学習に使えないので削る（学習のみ）
def remove_negative_session(df, _target):
    true_df = df.groupby('session')[_target].agg('sum') > 0
    session = pd.DataFrame(true_df[true_df]).reset_index()['session']
    df = df.merge(session, how = 'inner', on = 'session')
    return df

# 負例が多すぎる場合にunder samplingする
# ratio = pos/neg
def negative_sampling(df_x, df_y, ratio):
    print('before mean:', df_y.mean())

    Nrow = df_x.shape[0]
    Ndiv = 5
    n = int(Nrow // Ndiv) + 1

    df_x_list = [df_x.iloc[i*n : (i+1)*n, :] for i in range(Ndiv)]
    df_y_list = [df_y.iloc[i*n : (i+1)*n] for i in range(Ndiv)]
    del df_x, df_y
    gc.collect()

    for i in range(Ndiv):
        print('under sampling.......',i + 1 , '/', Ndiv)
        tmpx, tmpy = RandomUnderSampler(sampling_strategy=ratio, random_state=0).fit_resample(df_x_list[i], df_y_list[i])
        df_x_list[i] = tmpx
        df_y_list[i] = tmpy
        del tmpx, tmpy
        gc.collect()
    print('under sampling end')
    after_x = pd.concat(df_x_list)
    del df_x_list
    gc.collect()
    print('post proccess1')
    after_y = pd.concat(df_y_list)
    del df_y_list
    gc.collect()
    # sessionの順番がばらばらになるので再びsort
    tmp = pd.concat([after_x, after_y], axis=1).sort_values('session')
    after_y = tmp[target]
    after_x = tmp.drop(target , axis=1)

    print('after mean:', after_y.mean())
    return after_x, after_y

# dataframe, target_list, number of split data, number of fold
def add_cross_stacking_feature(df, cross_target_list, n_div, n_fold, base_path):
    # split data in order to save memory
    Nrow = df.shape[0]
    n = int(Nrow // n_div) + 1
    df_list = []
    for i in range(n_div):
        tmp = df.iloc[i*n : (i+1)*n, :]
        df_list.append(tmp)
    
    # initialization
    res_df = pd.DataFrame(columns=[], index = [])
    for i, t in enumerate(cross_target_list):
        df_pred = np.zeros(Nrow)
        for fold in range(n_fold):
            print('stacking target:', t, 'fold:', fold)
            model = np.load(f'{base_path}/otto/otto_lgbm_fold{fold}_{t}.pkl', allow_pickle=True)
            
            df_pred_list = []
            for i, v in enumerate(df_list):
                tmp = model.predict(v)
                df_pred_list.append(tmp)
            df_pred += np.concatenate(df_pred_list)
            
        tmp_df = pd.DataFrame(df_pred/n_fold, columns=[f'pred_stack_{t}'], dtype='float64')
        if i == 0:
            res_df = tmp_df
        else:
            res_df = pd.concat([res_df, tmp_df], axis=1)
            
    return pd.concat([df, res_df], axis=1)


In [6]:
# importanceが極端に低いものを削る (18件)
def remove_features(df):
    DROP_COL = ['session_type_mean']
    return df.drop(DROP_COL, axis=1)

In [7]:
train = fe.reduce_memory(train)

In [8]:
train = remove_negative_session(train, target)
print('target sum:', train[target].sum())
print('target mean:', train[target].mean())

target sum: 224413
target mean: 0.009848794650219957


# Hyperparameter Tuning by Optuna

In [9]:
# ====================================================
# Library
# ====================================================
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
# optuna
if OPTUNA_FLAG:
    import optuna.integration.lightgbm as lgb
else:
    import lightgbm as lgb

from itertools import combinations

In [10]:
if OPTUNA_FLAG:
    session = train['session']
    unique_session = session.unique()
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': [20],
        'boosting': 'gbdt',
        'seed': 42,        
        'n_jobs': -1,
        'learning_rate': 0.1
        }
    # Create a numpy array to store out of folds predictions
    N_splits = 5
    kfold = KFold(n_splits = N_splits, shuffle = True, random_state = 42)
    for fold, (trn_group_ind, val_group_ind) in enumerate(kfold.split(unique_session)):
        print(' ')
        print('-'*50)
        print(f'Training fold {fold}/{N_splits}....')
        # session単位で分割してKFoldする
        tr_groups, va_groups = unique_session[trn_group_ind], unique_session[val_group_ind]
        is_tr, is_va = session.isin(tr_groups), session.isin(va_groups)
        del tr_groups, va_groups
        gc.collect()
        # is_ir, is_va=Trueのindexを取得
        trn_ind, val_ind = is_tr[is_tr].index, is_va[is_va].index
        del is_tr, is_va
        gc.collect()

        y_train, y_val = train[target].iloc[trn_ind], train[target].iloc[val_ind]
        train_tmp = train.drop(IGNORE_COL_TARGET , axis=1)
        x_train, x_val = train_tmp.iloc[trn_ind], train_tmp.iloc[val_ind]
        del train_tmp
        gc.collect()

        # under sampling
        x_train, y_train = negative_sampling(x_train, y_train, pos_neg_ratio)

        # queryの準備, sessionごとにsortする, lightGBMでranking metricsを使うときに必要
        query_list_train = x_train['session'].value_counts()
        query_list_train = query_list_train.sort_index()

        query_list_valid = x_val['session'].value_counts()
        query_list_valid = query_list_valid.sort_index()
        

        # memory節約のため, under sampling後にfeature追加
        print('add session features....')
        x_train, x_val = fe.join_session_features(x_train, session_path), fe.join_session_features(x_val, session_path)
        print('add aid features....')
        x_train, x_val = fe.join_aid_features(x_train, aid_path), fe.join_aid_features(x_val, aid_path)
        print('add interactive features....')
        x_train, x_val = fe.join_interactive_features(x_train), fe.join_interactive_features(x_val)
        print('remove features....')
        x_train, x_val = remove_features(x_train),  remove_features(x_val)
        print('remove id from features....')
        x_train, x_val = x_train.drop(IGNORE_COL_ID, axis=1), x_val.drop(IGNORE_COL_ID, axis=1)
        print('x_train shape:', x_train.shape)

        lgb_train = lgb.Dataset(x_train, y_train, group=query_list_train)
        lgb_valid = lgb.Dataset(x_val, y_val, group=query_list_valid)

        del x_train, y_train
        gc.collect()

        #lgb_valid = lgb.Dataset(x_val, y_val)
        model = lgb.train(
            params = params,
            train_set = lgb_train,
            #num_boost_round = 10500,
            num_boost_round = 100,
            valid_sets = [lgb_train, lgb_valid],
            early_stopping_rounds = 20,
            verbose_eval = 10,
            )
        del lgb_train, lgb_valid
        gc.collect()
        break
    model.params

In [11]:
if OPTUNA_FLAG:
    print("Optuna results: ",model.params)

params = {'objective': 'lambdarank', 
          'metric': 'ndcg', 
          'ndcg_eval_at': [20], 
          'boosting': 'gbdt', 
          'seed': 42, 
          'n_jobs': -1, 
          #'learning_rate': 0.1, 
          'learning_rate': 0.05, 
          'feature_pre_filter': False, 
          'lambda_l1': 1.7510743847807332e-08, 
          'lambda_l2': 3.773149139134113e-07, 
          'num_leaves': 108, 
          'feature_fraction': 0.4, 
          'bagging_fraction': 1.0, 
          'bagging_freq': 0, 
          'min_child_samples': 20}

# Training

In [12]:

# Create a numpy array to store out of folds predictions
oof_predictions = np.zeros(len(train))
session = train['session']
unique_session = session.unique()

N_splits = 5
kfold = KFold(n_splits = N_splits, shuffle = True, random_state = 42)
for fold, (trn_group_ind, val_group_ind) in enumerate(kfold.split(unique_session)):
    print(' ')
    print('-'*50)
    print(f'Training fold {fold}/{N_splits}....')
    # session単位で分割してKFoldする
    tr_groups, va_groups = unique_session[trn_group_ind], unique_session[val_group_ind]
    is_tr, is_va = session.isin(tr_groups), session.isin(va_groups)
    del tr_groups, va_groups
    gc.collect()
    # is_ir, is_va=Trueのindexを取得
    trn_ind, val_ind = is_tr[is_tr].index, is_va[is_va].index
    del is_tr, is_va
    gc.collect()

    y_train, y_val = train[target].iloc[trn_ind], train[target].iloc[val_ind]
    train_tmp = train.drop(IGNORE_COL_TARGET , axis=1)
    x_train, x_val = train_tmp.iloc[trn_ind], train_tmp.iloc[val_ind]
    del train_tmp
    gc.collect()
    # under sampling
    x_train, y_train = negative_sampling(x_train, y_train, pos_neg_ratio)

    # queryの準備, sessionごとにsortする, lightGBMでranking metricsを使うときに必要
    query_list_train = x_train['session'].value_counts()
    query_list_train = query_list_train.sort_index()

    query_list_valid = x_val['session'].value_counts()
    query_list_valid = query_list_valid.sort_index()

    # memory節約のため, under sampling後にfeature追加
    print('add session features....')
    x_train, x_val = fe.join_session_features(x_train, session_path), fe.join_session_features(x_val, session_path)
    print('add aid features....')
    x_train, x_val = fe.join_aid_features(x_train, aid_path), fe.join_aid_features(x_val, aid_path)
    print('add interactive features....')
    x_train, x_val = fe.join_interactive_features(x_train), fe.join_interactive_features(x_val)
    print('remove features....')
    x_train, x_val = remove_features(x_train),  remove_features(x_val)
    print('remove id from features....')
    x_train, x_val = x_train.drop(IGNORE_COL_ID, axis=1), x_val.drop(IGNORE_COL_ID, axis=1)
    print('x_train shape:', x_train.shape)
    
    if CROSS_TARGET_STACKING:
        print('cross_feature stacking, x_train...')
        x_train = add_cross_stacking_feature(x_train, cross_target_list, n_div=5, n_fold=5, base_path=base_path)
        print('cross_feature stacking, x_val...')
        x_val = add_cross_stacking_feature(x_val, cross_target_list, n_div=5, n_fold=5, base_path=base_path)
        
    lgb_train = lgb.Dataset(x_train, y_train, group=query_list_train)
    lgb_valid = lgb.Dataset(x_val, y_val, group=query_list_valid)

    del x_train, y_train
    gc.collect()

    model = lgb.train(
        params = params,
        train_set = lgb_train,
        #num_boost_round = 100,
        num_boost_round = 2000,
        valid_sets = [lgb_train, lgb_valid],
        early_stopping_rounds = 20,
        verbose_eval = 10,
        )
    del lgb_train, lgb_valid
    gc.collect()

    # Save best model
    
    if CROSS_TARGET_STACKING:
        joblib.dump(model, f'{base_path}/otto/otto_lgbm_fold{fold}_{TYPE_MODE}_stack.pkl')
    else:
        joblib.dump(model, f'{base_path}/otto/otto_lgbm_fold{fold}_{TYPE_MODE}.pkl')
    # Predict validation
    # でかいので分割してpredict
    Nrow = x_val.shape[0]
    Ndiv = 5
    n = int(Nrow // Ndiv) + 1
    x_val_list = []
    for i in range(Ndiv):
        tmp = x_val.iloc[i*n : (i+1)*n, :]
        x_val_list.append(tmp)
    del x_val
    gc.collect()

    val_pred_list = []
    for i, v in enumerate(x_val_list):
        print('train pred i=', i)
        tmp = model.predict(v)
        val_pred_list.append(tmp)
    del x_val_list
    gc.collect()
    val_pred = np.concatenate(val_pred_list)
    del val_pred_list
    gc.collect()

    # Add to out of folds array
    # CVを終えれば全部のindexが1回ずつ計算されることになる
    oof_predictions[val_ind] = val_pred

    # 不要になった時点でモデル削除
    del model, y_val
    gc.collect()

    # tmp recall for each fold
    df = pd.DataFrame(val_pred, columns=["score"])
    tmp = train[['session', 'aid']].iloc[val_ind].reset_index(drop=True)
    pred_df = pd.concat([tmp, df], axis=1)
    del tmp
    gc.collect()

    pred_df['session_type'] = pred_df['session'].apply(lambda x: str(x) + f'_{TYPE_MODE}')
    pred_df = pred_df.sort_values(['session_type','score'],ascending=[True, False]).reset_index(drop=True)

    pred_df['n'] = pred_df.groupby('session_type').cumcount()
    pred_df = pred_df.loc[pred_df.n<20].drop(['n','score','session'],axis=1)
    pred_df['aid'] = pred_df['aid'].astype('int32')
    pred_df = pred_df.groupby('session_type')['aid'].apply(list).reset_index()
    pred_df['labels'] = pred_df['aid'].map(lambda x: ''.join(str(x)[1:-1].split(',')))
    pred_df = pred_df.drop(['aid'],axis=1)

    sub = pred_df.loc[pred_df.session_type.str.contains(TYPE_MODE)].copy()
    sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

    test_labels = pd.read_parquet(f'{base_path}/input/otto/otto-validation/test_labels.parquet')
    test_labels = test_labels.loc[test_labels['type']==TYPE_MODE]
    # foldごとのreallなのでinnter
    test_labels = test_labels.merge(sub, how='inner', on=['session']) 
    test_labels['labels'] = test_labels['labels'].fillna('[]')
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print(f'fold {fold} {TYPE_MODE} recall =',recall)


 
--------------------------------------------------
Training fold 0/5....
before mean: 0.009829630132793749
under sampling....... 1 / 5
under sampling....... 2 / 5
under sampling....... 3 / 5
under sampling....... 4 / 5
under sampling....... 5 / 5
under sampling end
post proccess1
after mean: 0.025
add session features....
add aid features....
add interactive features....
remove features....
remove id from features....
x_train shape: (7165960, 173)
cross_feature stacking, x_train...
stacking target: carts fold: 0
stacking target: carts fold: 1
stacking target: carts fold: 2
stacking target: carts fold: 3
stacking target: carts fold: 4
stacking target: clicks fold: 0
stacking target: clicks fold: 1
stacking target: clicks fold: 2
stacking target: clicks fold: 3
stacking target: clicks fold: 4
cross_feature stacking, x_val...
stacking target: carts fold: 0
stacking target: carts fold: 1
stacking target: carts fold: 2
stacking target: carts fold: 3
stacking target: carts fold: 4
stacking

[170]	training's ndcg@20: 0.901317	valid_1's ndcg@20: 0.827325
[180]	training's ndcg@20: 0.902242	valid_1's ndcg@20: 0.827323
[190]	training's ndcg@20: 0.902976	valid_1's ndcg@20: 0.827465
[200]	training's ndcg@20: 0.903669	valid_1's ndcg@20: 0.827464
[210]	training's ndcg@20: 0.904279	valid_1's ndcg@20: 0.827559
[220]	training's ndcg@20: 0.904983	valid_1's ndcg@20: 0.827697
[230]	training's ndcg@20: 0.905653	valid_1's ndcg@20: 0.827723
[240]	training's ndcg@20: 0.906407	valid_1's ndcg@20: 0.82766
[250]	training's ndcg@20: 0.907184	valid_1's ndcg@20: 0.827645
Early stopping, best iteration is:
[232]	training's ndcg@20: 0.905831	valid_1's ndcg@20: 0.827786
train pred i= 0
train pred i= 1
train pred i= 2
train pred i= 3
train pred i= 4
fold 2 orders recall = 0.7321093845803479
 
--------------------------------------------------
Training fold 3/5....
before mean: 0.009842684972505789
under sampling....... 1 / 5
under sampling....... 2 / 5
under sampling....... 3 / 5
under sampling.......

In [13]:
df = pd.DataFrame(oof_predictions, columns=["score"])
pred_df = pd.concat([train[['session', 'aid']], df], axis=1)
pred_df['session_type'] = pred_df['session'].apply(lambda x: str(x) + f'_{TYPE_MODE}')
pred_df = pred_df.sort_values(['session_type','score'],ascending=[True, False]).reset_index(drop=True)

if CROSS_TARGET_STACKING:
    pred_df.to_parquet(f'{base_path}/otto/oof_lgbm_{TYPE_MODE}_stack.parquet')
else:
    pred_df.to_parquet(f'{base_path}/otto/oof_lgbm_{TYPE_MODE}.parquet')

pred_df['n'] = pred_df.groupby('session_type').cumcount()
pred_df = pred_df.loc[pred_df.n<20].drop(['n','score','session'],axis=1)
pred_df['aid'] = pred_df['aid'].astype('int32')
pred_df = pred_df.groupby('session_type')['aid'].apply(list).reset_index()
pred_df['labels'] = pred_df['aid'].map(lambda x: ''.join(str(x)[1:-1].split(',')))
pred_df = pred_df.drop(['aid'],axis=1)
pred_df

,session_type,labels
0,11098528_orders,11830 1732105 588923 876129 884502 1157882 571...
1,11098530_orders,409236 1603001 264500 364155 963957 254154 210...
2,11098531_orders,1365569 1557766 1309633 1553691 1449555 396199...
3,11098533_orders,1074173 1309900 1165015 765030 833149 1622419 ...
4,11098534_orders,223062 908024 1607945 1342293 1449202 530377 1...
...,...,...
133618,12899159_orders,1512596 1383649 1131172 314450 1178395 1616589...
133619,12899329_orders,1333457 1470364 356732 1667554 931182 977011 1...
133620,12899337_orders,558573 1662401 742581 1223508 1581401 1600937 ...
133621,12899373_orders,1766353 487949 461938 1662986 1763592 516917 1...


In [14]:
sub = pred_df.loc[pred_df.session_type.str.contains(TYPE_MODE)].copy()
sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])

test_labels = pd.read_parquet(f'{base_path}/input/otto/otto-validation/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==TYPE_MODE]
test_labels = test_labels.merge(sub, how='left', on=['session'])
test_labels['labels'] = test_labels['labels'].fillna('[]')
test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
print(f'{TYPE_MODE} recall =',recall)

orders recall = 0.6647973367634527


In [15]:
# click total: 1,755,534
# 0.52なら912,877の正解が必要

In [ ]:
# ranker model, fold0を factor=1.111369 で割ればそれっぽい値が出る, each foldは session inner joinなので高めに出る
# num=100, lr=0.1, no feature add, valid_1's ndcg@50: 0.843586, fold 0 orders recall = 0.7263385039885385, orders recall = 0.6535526311589739
# add aid feature, valid_1's ndcg@50: 0.84962, fold 0 orders recall = 0.7305304490864389, (orders recall = 0.657324?)
# add aid + session feature, valid_1's ndcg@50: 0.849762, fold 0 orders recall = 0.7302651361055592, orders recall = 0.6575806806829172
# all valid_1's ndcg@50: 0.848664, fold 0 orders recall = 0.730990324919964, orders recall = 0.6579892308723504


# hypter paramやりなおし、regularization param大幅変更 num=100, lr=0.1
# no add: valid_1's ndcg@20: 0.835401, fold 0 orders recall = 0.7261793162000106, orders recall = 0.6535877409408783  -> order,carts差し替えPB = 0.581
#                                                                                 carts recall = 0.41837386076234817
# sessionのみ: valid_1's ndcg@20: 0.836164, fold 0 orders recall = 0.7268514424182394, orders recall = 0.6545069788671031 -> order,carts差し替えPB = 0.583
#                                                                                      carts recall = 0.4196106730132077
# aidのみ: valid_1's ndcg@20: 0.842205, fold 0 orders recall = 0.7302828236376179, orders recall = 0.6575838724812721 -> order,carts差し替えPB = 0.586 (55, 1/19)
#                                                                                  carts recall = 0.42261163401459195                              
# aid+session: valid_1's ndcg@20: 0.843169, fold 0 orders recall = 0.7309018872596706, orders recall = 0.6582030813621319 -> order,carts差し替えPB = 0.587 (54, 1/19)
#                                                                                      carts recall = 0.42347896378377814
# all: valid_1's ndcg@20: 0.843514, fold 0 orders recall = 0.731184887772609, orders recall = 0.6584137400535583 -> order,carts差し替えPB = 0.586 下がったけどブレ？
#                                                                             carts recall = 0.42349804503870025
# num=1000, lr=0.05 [281] valid_1's ndcg@20: 0.844737, fold 0 orders recall = 0.7315386384137821, orders recall = 0.6586627003252442 -> PB = 0.587 (53, 1/19)
#                                                                                                 carts recall = 0.4242057861303562
#                                                                                                 clicks recall = 0.536971
# candidate add, mean 60 -> 120
# lr=0.1, [100] valid_1's ndcg@20: 0.832289 fold 0 orders recall = 0.7260696029689797, orders recall = 0.6619853624127442-> PB = 0.592 
# lr=0.05,[172] valid_1's ndcg@20: 0.833279,fold 0 orders recall = 0.7268398571528605, orders recall = 0.6622981586515291
#                                                                                      carts recall = 0.4296682290166909
#                                                                                      clicks recall = 0.54153778850196010
# candidate add more
# lr=0.05,[172] valid_1's ndcg@20: 0.824422, fold 0 orders recall = 0.7264196759981961, orders recall = 0.6627609694129963-> PB = 0.592
#                                                                                       carts recall = 0.43018862687820264
#                                                                                       clicks recall = 0.5425910292822583
# stacking test, be careful to leak..
# lr=0.1,  valid_1's ndcg@20: 0.825827 [100] fold 0 orders recall = 0.7276858500711139, orders recall = 0.6645771026769612
# lr=0.05 valid_1's ndcg@20: 0.826031 [131] fold 0 orders recall = 0.728258230131474, orders recall = 0.6647973367634527


